In [ ]:
# Install packages
!pip install nltk
!pip install sentence-transformers
!pip install torch torchvision torchaudio

Possbilità anche di utilizzare un tokenizzatore grazie a spacy o NLTK

In [ ]:
import nltk
import re
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
#from transformers import AutoTokenizer, AutoModel

#tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
#model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

nltk.download('stopwords')  # Download nltk stopwords
nltk.download('punkt')      # Download nltk punkt tokenizer
#nltk.download('wordnet')

text = []
stopwords_list = set(stopwords.words("english"))

file_path = r'C:\Users\Utente\Desktop\Semantic Textual Similarity\Clinical Semantic Textual Similarity\clinicalSTS2019\train\clinicalSTS2019.train.txt'


with open(file_path, 'r') as f:
    for line in f.readlines():

        # Splitting by tab
        line_parts = re.split('\t', line)
        
        filtered_sentences = []
        for sentence in line_parts:
            # Lowercasing
            lower_sentence = sentence.lower()

            remove_apex = lower_sentence.replace ('"', ' ')
            
            remove_whitespace = remove_apex.strip()
            
            remove_punteggiatura = re.sub(r'[^\w\s.]', '', remove_whitespace)
            
            rimuovi_puntifin = re.sub(r'\.$', '', remove_punteggiatura)
            
            #tokenized_sentence = word_tokenize(rimuovi_puntifin)
            
            # Remove stopwords
            filtered_tokens = [token for token in rimuovi_puntifin if token.lower() not in stopwords_list]

            filtered_sentences.append(filtered_tokens)
            
        # Conversione del terzo elemento in un numero
        filtered_sentences[-1] = float(filtered_sentences[-1][0])
        
        text.append(filtered_sentences)
        

In [ ]:
text[0]

Viene inserito la tokenizzazione utilizzando un modello preaddestrato. Non restituisce risultati sperati. Non viene eseguita alcuna tecnica di normalizzazione (lemmatizzazione o stemming)

In [ ]:
def make_split(embedded_representation, labels):
    X_train, X_test, y_train, y_test = train_test_split(embedded_representation, labels, random_state=104, test_size=0.25, shuffle=True)

    y_train = np.array(y_train, dtype=object)
    y_test = np.array(y_test, dtype=object)

    X_train = np.array(X_train, dtype=object)
    X_test = np.array(X_test, dtype=object)

    return X_train, X_test, y_train, y_test

In [ ]:
def cosine_similarity_unidimensional(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

In [ ]:
def normalize_labels(label):
    label = [float(val) for val in label]  # Converti i valori in numeri decimali
    min_val = np.min(label)
    max_val = np.max(label)
    normalized_labels_ = (label - min_val) / (max_val - min_val) * 2 - 1

    return normalized_labels_

In [ ]:
text_embedded=[]
label=[]

model=SentenceTransformer('bert-base-nli-mean-tokens')

for sample in range(len(text)):
    sentence_vectors = model.encode(text[sample][0:2])
    text_embedded.append(sentence_vectors)
    label.append(text[sample][2])

labels = normalize_labels(label)

In [ ]:
similarities = []

for i in range(len(text_embedded)):
    embedding_pair = text_embedded[i]
    similarity = cosine_similarity_unidimensional([embedding_pair[0]], [embedding_pair[1]])
    similarities.append(similarity[0][0])


In [ ]:
correlation, p_value = pearsonr(similarities, labels)

print("Correlation coefficient:", correlation)
print("p-value:", p_value)